## 1.Import Library

In [1]:
import os
import pdfplumber
import spacy
import re
from unidecode import unidecode
import pandas as pd

###

## 2.Build algorithms

### 2.1.Import model from Spacy

In [2]:
# Tải mô hình NER từ SpaCy
nlp = spacy.load("en_core_web_lg")

### 2.2.Preprocessing of text data

#### 2.2.1.Set the rules

> List of common Vietnamese surnames

In [3]:
family_names = [
    'nguyen', 'tran', 'le', 'pham', 'vu', 'hoang', 'ngo', 'dang', 'bui', 'duong',
    'do', 'ha', 'dao', 'ly', 'ho', 'cao', 'van', 'to',
    'bach', 'chau', 'chung', 'luong', 'lam', 'loc', 'nghiem',
    'quang', 'phan', 'vo', 'son', 'hong', 'khanh', 'mac', 'hoai',
    'tang', 'ta', 'nghiem', 'phu', 'luc', 'quyen', 'tong', 'mau', 'hanh',
    'giang', 'tu', 'ba', 'phuong', 'khưu', 'diep', 'ham', 'dinh', 'phuc',
    'hanh', 'mai', 'lien', 'nho', 'ton', 'ngoc',
    'a', 'au', 'bao', 'bang', 'bac', 'ban', 'cat', 'can', 'cang',
    'cuong', 'co', 'cu', 'dao', 'doan', 'dinh', 'dau', 'duy', 'dien',
    'gia', 'hoai', 'hoa', 'hung', 'huynh', 'hue', 'khu', 'kim', 'kien',
    'kinh', 'la', 'lan', 'lien', 'mon', 'man', 'minh', 'nghia', 'nhan',
    'ninh', 'nham', 'nhat', 'pho', 'quach', 'quan', 'quang', 'quyen',
    'ton', 'tu', 'tong', 'van', 'vo', 'vu'
]


> Function remove accents

In [4]:
def remove_accents(text):
    return unidecode(text)

Function check name VietNam

In [5]:
def is_vietnamese_name(name):
    name_no_accents = remove_accents(name.lower())
    parts = name_no_accents.split()
    surname = parts[0]
    return surname in family_names

> Function extract data from PDF

In [6]:
def extract_text_from_pdf(pdf_path, lines_to_read=20):
    text = ""
    if not os.path.isfile(pdf_path):
        raise FileNotFoundError(f"{pdf_path} không tồn tại hoặc không phải là tệp.")
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(min(len(pdf.pages), 2)):  
            page = pdf.pages[i]
            lines = page.extract_text().splitlines()
            for line in lines[:lines_to_read]:
                text += line + "\n"
    return text

> Function extract Name,Email from PDF

In [7]:
def extract_names_and_emails(text):
    text_no_accents = remove_accents(text)
    doc = nlp(text_no_accents)
    
    # Trích xuất tên từ SpaCy
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    filtered_names = []
    for name in names:
        if len(name.split()) >= 2 and is_vietnamese_name(name):
            filtered_names.append(name)
            break 
    
    # Trích xuất email và loại bỏ trùng lặp
    emails = list(set(re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', text_no_accents)))
    
    return filtered_names, emails


>Function Process pdfs

In [8]:
# Hàm xử lý nhiều tệp PDF trong thư mục
def process_pdfs_in_directory(directory):
    seen_emails = set()  # Tập hợp để lưu trữ các email đã nhận dạng
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory, filename)
            text = extract_text_from_pdf(pdf_path)
            names, emails = extract_names_and_emails(text)
            
            # Chỉ in các email chưa từng xuất hiện
            unique_emails = [email for email in emails if email not in seen_emails]
            seen_emails.update(unique_emails)
            
            print(f"Tệp: {filename}")
            print(f"Name: {names if names else 'Không tìm thấy tên'}")
            print(f"Email: {unique_emails if unique_emails else 'Không tìm thấy email'}\n")


> Function process single_pdf

In [9]:
def process_single_pdf(pdf_path):
    if not os.path.isfile(pdf_path):
        raise FileNotFoundError(f"{pdf_path} không tồn tại hoặc không phải là tệp.")
        
    text = extract_text_from_pdf(pdf_path)
    names, emails = extract_names_and_emails(text)
    
    print(f"Tệp: {os.path.basename(pdf_path)}")
    print(f"Name: {names if names else 'Không tìm thấy tên'}")
    print(f"Email: {emails if emails else 'Không tìm thấy email'}\n")


> Single_pdf

In [10]:
process_single_pdf("../data/resumes/android_02.pdf")

Tệp: android_02.pdf
Name: ['Hoang Van Duong']
Email: ['duonghv@fsoft.com.vn']



>  Multiple file recognition

In [11]:
process_pdfs_in_directory("../data/resumes")

Tệp: android_01.pdf
Name: ['Nguyen Thanh Xuan']
Email: ['xuanusm@gmail.com']

Tệp: android_02.pdf
Name: ['Hoang Van Duong']
Email: ['duonghv@fsoft.com.vn']

Tệp: android_03.pdf
Name: ['Tang Nhon']
Email: ['kimtungcdt@gmail.com']

Tệp: android_04.pdf
Name: Không tìm thấy tên
Email: Không tìm thấy email

Tệp: android_05.pdf
Name: ['Ho Chi Minh City']
Email: ['me@toan.mobi']

Tệp: android_06.pdf
Name: ['Nguyen Dang Phat']
Email: ['phatnd89@gmail.com']

Tệp: android_07.pdf
Name: ['Tran Address']
Email: ['cao.tranchi@gmail.com']

Tệp: android_09.pdf
Name: Không tìm thấy tên
Email: ['dohoangyen93@gmail.com']

Tệp: android_10.pdf
Name: ['Huynh Thai Tam\nComputers']
Email: ['thaitam.fetel.2406@gmail.com']

Tệp: android_11.pdf
Name: ['Ho Chi Minh City']
Email: ['anhtuanbd84@gmail.com']

Tệp: android_12.pdf
Name: ['Hoang Thanh Tung']
Email: ['tung.hoangthanh90@gmail.com']

Tệp: android_13.pdf
Name: ['HUYNH VAN TOAN']
Email: Không tìm thấy email

Tệp: android_14.pdf
Name: ['NGUYEN CONG']
Email: [

KeyboardInterrupt: 